In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd '/content/drive/MyDrive/'

/content/drive/MyDrive


In [ ]:
dev_relaxed = pd.read_csv('uqa_relaxed_dev.csv')
test_relaxed = pd.read_csv('uqa_relaxed_test.csv')

dev_strict = pd.read_csv('uqa_strict_dev.csv')
test_strict = pd.read_csv('uqa_strict_test.csv')

In [ ]:
dev_relaxed.columns

Index(['question-X', 'answer-Y', 'goldstandard2', 'uqa_input', 'data_type',
       'predictions'],
      dtype='object')

In [ ]:
print('Overall Accuracy Relaxed:',test_relaxed['goldstandard2'].eq(test_relaxed['predictions']).mean())

Overall Accuracy Relaxed: 0.8964994696166085


In [ ]:
print('Overall Accuracy Strict:',test_strict['goldstandard1'].eq(test_strict['predictions']).mean())

Overall Accuracy Strict: 0.7474210186976145


In [ ]:
def groupwise_acc(df, answer):
  return df[answer].eq(df['predictions']).groupby(df[answer]).mean()

In [ ]:
names = ['Relaxed Dev', 'Relaxed Test', 'Strict Dev', 'Strict Test']
dfs = [dev_relaxed, test_relaxed, dev_strict, test_strict]
answers = ['goldstandard2', 'goldstandard2', 'goldstandard1', 'goldstandard1']

for df, name, answer in zip(dfs, names, answers):
  print(name+'\n-----------')
  print(groupwise_acc(df, answer))
  print("======================\n")


Relaxed Dev
-----------
goldstandard2
in the middle, neither yes nor no    0.205263
no                                   0.929462
yes                                  0.911305
yes, subject to some conditions      0.847195
dtype: float64

Relaxed Test
-----------
goldstandard2
in the middle, neither yes nor no    0.278947
no                                   0.931048
yes                                  0.914612
yes, subject to some conditions      0.835271
dtype: float64

Strict Dev
-----------
goldstandard1
i am not sure how x will interpret y’s answer    0.000000
in the middle, neither yes nor no                0.000000
no                                               0.696214
probably no                                      0.038793
probably yes / sometimes yes                     0.184739
yes                                              0.907273
yes, subject to some conditions                  0.854932
dtype: float64

Strict Test
-----------
goldstandard1
i am not sure how x will i

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat)

In [ ]:
from sklearn.metrics import f1_score

print(f1_score(dev_relaxed['goldstandard2'], dev_relaxed['predictions'], average=None))
print(f1_score(test_relaxed['goldstandard2'], test_relaxed['predictions'], average=None))
print(f1_score(dev_strict['goldstandard1'], dev_strict['predictions'], average=None))
print(f1_score(test_strict['goldstandard1'], test_strict['predictions'], average=None))


[0.31967213 0.89847429 0.90952738 0.89387755]
[0.4140625  0.90086694 0.91214393 0.89233954]
[0.         0.         0.73132881 0.07114625 0.28930818 0.80243902
 0.87093596]
[0.         0.03100775 0.7334449  0.04819277 0.26923077 0.80368947
 0.8633235 ]


In [ ]:
dev_relaxed.groupby(dev_relaxed['goldstandard2']).groups.keys()

dict_keys(['in the middle, neither yes nor no', 'no', 'yes', 'yes, subject to some conditions'])

In [ ]:
for df, name, answer in zip(dfs, names, answers):
  print(name+'\n-----------')
  labels = list(df.groupby(df[answer]).groups.keys())
  scores = f1_score(df[answer], df['predictions'], average=None)
  res = list(zip(labels, scores))
  for i in res:
    print(i)
  print("======================\n")


Relaxed Dev
-----------
('in the middle, neither yes nor no', 0.31967213114754095)
('no', 0.898474288943304)
('yes', 0.9095273818454612)
('yes, subject to some conditions', 0.8938775510204081)

Relaxed Test
-----------
('in the middle, neither yes nor no', 0.4140625)
('no', 0.9008669430833018)
('yes', 0.9121439280359821)
('yes, subject to some conditions', 0.8923395445134575)

Strict Dev
-----------
('i am not sure how x will interpret y’s answer', 0.0)
('in the middle, neither yes nor no', 0.0)
('no', 0.7313288069835112)
('probably no', 0.07114624505928854)
('probably yes / sometimes yes', 0.2893081761006289)
('yes', 0.802439024390244)
('yes, subject to some conditions', 0.870935960591133)

Strict Test
-----------
('i am not sure how x will interpret y’s answer', 0.0)
('in the middle, neither yes nor no', 0.031007751937984496)
('no', 0.7334448960076501)
('probably no', 0.048192771084337345)
('probably yes / sometimes yes', 0.2692307692307692)
('yes', 0.8036894696387396)
('yes, subject

In [ ]:
print(f1_score(dev_relaxed['goldstandard2'], dev_relaxed['predictions'], average='weighted'))
print(f1_score(test_relaxed['goldstandard2'], test_relaxed['predictions'], average='weighted'))
print(f1_score(dev_strict['goldstandard1'], dev_strict['predictions'], average='weighted'))
print(f1_score(test_strict['goldstandard1'], test_strict['predictions'], average='weighted'))

0.887020078230354
0.8918677417050246
0.7172846675279307
0.7169208399990962


In [ ]:
test_relaxed[test_relaxed['goldstandard2']!= test_relaxed['predictions']]

,question-X,answer-Y,goldstandard2,uqa_input,data_type,predictions
8,is it an office job?,it's an assistant position.,"in the middle, neither yes nor no",is it an office job? \n (A) yes (B) no (C) in ...,test,no
12,do you own property already?,i would prefer to pay a mortgage more than rent.,"in the middle, neither yes nor no",do you own property already? \n (A) yes (B) no...,test,no
13,do you like fantasy novels?,disney is cool.,"in the middle, neither yes nor no",do you like fantasy novels? \n (A) yes (B) no ...,test,yes
28,does it feel strange to see me again?,if feels great to see you again.,no,does it feel strange to see me again? \n (A) y...,test,yes
29,is it the end of the day already?,it's hard to believe.,yes,is it the end of the day already? \n (A) yes (...,test,no
...,...,...,...,...,...,...
6571,can you afford the flat?,i'm already preapproved for a mortgage.,yes,can you afford the flat? \n (A) yes (B) no (C)...,test,no
6581,are you a fan of electronic music?,synthesizers are funky,no,are you a fan of electronic music? \n (A) yes ...,test,yes
6585,do you still live around here?,i've just moved back.,yes,do you still live around here? \n (A) yes (B) ...,test,no
6588,have you watched disney movies?,i haven't seen any lately,yes,have you watched disney movies? \n (A) yes (B)...,test,no


In [ ]:
test_relaxed[test_relaxed['goldstandard2']!= test_relaxed['predictions']]['predictions'].value_counts()

no                                   349
yes                                  302
yes, subject to some conditions       19
in the middle, neither yes nor no     13
Name: predictions, dtype: int64

In [30]:
test_strict[test_strict['goldstandard1']!= test_strict['predictions']].head(20)

,question-X,answer-Y,goldstandard1,uqa_input,data_type,predictions
9,was the travel tiring?,it was exciting,no,was the travel tiring? \n (A) yes (B) no (C) i...,test,yes
12,do you like food that are primarily barbecued?,it is okay to have sometimes,probably yes / sometimes yes,do you like food that are primarily barbecued?...,test,yes
19,want to go to the cinema?,i'd prefer to see the sights.,probably no,want to go to the cinema? \n (A) yes (B) no (C...,test,no
20,are you a fan of classic literature?,i think i have read much of what would be cons...,"in the middle, neither yes nor no",are you a fan of classic literature? \n (A) ye...,test,no
22,want to work for another company?,i want to start my own business.,no,want to work for another company? \n (A) yes (...,test,yes
32,are you into the outdoors?,i prefer it to being indoors,yes,are you into the outdoors? \n (A) yes (B) no (...,test,no
40,are you looking to meet people?,i'm happy on my own.,no,are you looking to meet people? \n (A) yes (B)...,test,yes
43,do you listen to new music every friday?,i listen to the radio daily.,probably yes / sometimes yes,do you listen to new music every friday? \n (A...,test,yes
45,do you want to go swimming?,i'd rather go skating.,probably no,do you want to go swimming? \n (A) yes (B) no ...,test,no
48,have you started looking yet?,i'll look when we arrive.,no,have you started looking yet? \n (A) yes (B) n...,test,yes


In [ ]:
test_strict[test_strict['goldstandard1']!= test_strict['predictions']]['predictions'].value_counts()

yes                                990
no                                 483
yes, subject to some conditions     62
probably yes / sometimes yes        21
probably no                         11
Name: predictions, dtype: int64

In [ ]:
full = pd.read_csv('NLU_Project/circa-data.tsv', sep='\t', index_col='id')


In [ ]:
full['goldstandard1'].value_counts()

Yes                                              14504
No                                               10829
Yes, subject to some conditions                   2583
Probably yes / sometimes yes                      1244
Probably no                                       1160
In the middle, neither yes nor no                  638
Other                                              504
I am not sure how X will interpret Y’s answer       63
Name: goldstandard1, dtype: int64

In [ ]:
full['goldstandard1'].value_counts(normalize = True)

Yes                                              0.460079
No                                               0.343505
Yes, subject to some conditions                  0.081935
Probably yes / sometimes yes                     0.039461
Probably no                                      0.036796
In the middle, neither yes nor no                0.020238
Other                                            0.015987
I am not sure how X will interpret Y’s answer    0.001998
Name: goldstandard1, dtype: float64

In [ ]:
full['goldstandard2'].value_counts()

Yes                                  16628
No                                   12833
Yes, subject to some conditions       2583
In the middle, neither yes nor no      949
Other                                  504
Name: goldstandard2, dtype: int64

In [ ]:
full['goldstandard2'].value_counts(normalize = True)

Yes                                  0.496403
No                                   0.383109
Yes, subject to some conditions      0.077111
In the middle, neither yes nor no    0.028331
Other                                0.015046
Name: goldstandard2, dtype: float64